<a href="https://colab.research.google.com/github/stsan9/AnomalyDetection4Jets/blob/master/Machine_Learning_Model_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import math
from random import sample 
import statistics
from datetime import datetime

In [0]:
# Mount the google drive file system
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
# load dataset
path = '/content/gdrive/My Drive/McAuley/NewUserTimeTable.csv'
user_time_table = pd.read_csv(path)
user_time_table.head(2)

,userId,user_median,num_recent_workouts,recent_sport,workout_soon,recent_heart_rate,recent_speed,recent_distance,recent_altitude,months_to_next1,gender_female,gender_male,gender_unknown,hrs_to_next1,hrs_to_next2,months_to_next2
0,69,83.720278,1,1,0.0,113.518,6.193348,7.062137,14.3804,0.909113,0,1,0,654.561389,786.826111,1.092814
1,2358,29.547778,1,3,0.0,169.216,15.791680,7.601939,132.9952,0.017045,0,1,0,12.272500,204.958611,0.284665


# No need to run the code below

In [0]:
def time_scale(num):
  return num/60/60

In [0]:
all_users=dataframe["userId"].unique()
saved_stamps = list(dataframe['time_start'])
times = []
for user in all_users:
  user_data = dataframe.loc[dataframe["userId"] == user].sort_values("time_start")
  arr = np.diff(user_data["time_start"])
  arr=np.append(arr,[0])
  user_data["time_start"]=arr.tolist()
  indexNames = dataframe[dataframe["userId"] == user].index
  dataframe.drop(indexNames,inplace=True)
  dataframe = dataframe.append(user_data,ignore_index = True)

dataframe = dataframe.rename( columns= { "time_start" : "hrs_to_next" } )
dataframe['time_start'] = saved_stamps
dataframe['hrs_to_next'] = dataframe['hrs_to_next'].apply(time_scale)

In [0]:
df = dataframe.copy()
df = df[df['hrs_to_next'] != 0]
median_gap = np.median(df['hrs_to_next'])
print(median_gap/24)

1.9372280092592593


In [0]:
#Converting hrs_to_next to month ranges
def hrs_to_month(num):
  return num/24/30
dataframe["months_to_next"]=dataframe["hrs_to_next"].apply(hrs_to_month)

In [0]:
#Converting time_start to number of month
def extract_month(num):
  date=datetime.fromtimestamp(num)
  return date.month
dataframe["month"]=dataframe["time_start"].apply(extract_month)

In [0]:
user_time_table = pd.DataFrame(columns=['userId', 'median_hrs_to_next', 'workout_in_median'])

variable_dict = {'user_median': [], 'recent_workouts': [], 'workout_soon': [], 'recent_sport': [], 'recent_heart_rate': [], 'recent_speed': [], 'recent_distance': [],
                 'recent_altitude': [], 'good_users': [], 'hrs_to_next1': [], 'hrs_to_next2': [], 'gender_female': [], 'gender_male': [], 'gender_unknown': [], 'months_to_next1': [],
                 'months_to_next2': [], 'recent_month1': [], 'recent_month2': [], 'time_int_deviation': []}

for user in all_users:
  user_data = dataframe.loc[dataframe["userId"] == user].loc[dataframe['hrs_to_next'] != 0].sort_values("time_start", ascending=False)
  user_data['time_int_deviation'] = pd.DataFrame.std(user_data.hrs_to_next)   # calculate time deviation per user here
  if len(user_data) >= 20:
    variable_dict['good_users'].append(user)
    variable_dict['user_median'].append(np.median(user_data['hrs_to_next']))
    workouts = user_data[user_data['time_start'] >= (user_data['time_start'].iloc[1] - median_gap*60*60)]
    variable_dict['recent_workouts'].append(len(workouts) - 2)
    variable_dict['recent_sport'].append(user_data['sport'].iloc[1])
    variable_dict['workout_soon'].append(int(user_data['hrs_to_next'].iloc[1] <= median_gap))
    variable_dict['recent_heart_rate'].append(user_data['average_heart_rate'].iloc[1])
    variable_dict['recent_speed'].append(user_data['dervided_speed (km/h)'].iloc[1])
    variable_dict['recent_distance'].append(user_data['derived_distance (km)'].iloc[1])
    variable_dict['recent_altitude'].append(user_data['average_altitude'].iloc[1])
    variable_dict['hrs_to_next1'].append(user_data['hrs_to_next'].iloc[2])
    variable_dict['hrs_to_next2'].append(user_data['hrs_to_next'].iloc[3])
    variable_dict['months_to_next1'].append(user_data['months_to_next'].iloc[2])
    variable_dict['months_to_next2'].append(user_data['months_to_next'].iloc[3])
    variable_dict['gender_female'].append(user_data['gender_female'].iloc[1])
    variable_dict['gender_male'].append(user_data['gender_male'].iloc[1])
    variable_dict['gender_unknown'].append(user_data['gender_unknown'].iloc[1])
    variable_dict['recent_month1'].append(user_data['month'].iloc[2])
    variable_dict['recent_month2'].append(user_data['month'].iloc[3])
    variable_dict['time_int_deviation'].append(user_data['time_int_deviation'].iloc[1])

user_time_table = pd.DataFrame.from_dict(variable_dict)

In [0]:
user_time_table.to_csv(r'/content/gdrive/My Drive/McAuley/UserTimeTable.csv', index = False)

# Table


In [0]:
# DataFrame table
user_time_table

,userId,user_median,num_recent_workouts,recent_sport,workout_soon,recent_heart_rate,recent_speed,recent_distance,recent_altitude,months_to_next1,gender_female,gender_male,gender_unknown,hrs_to_next1,hrs_to_next2,months_to_next2
0,69,83.720278,1,1,0.0,113.518,6.193348,7.062137,14.3804,0.909113,0,1,0,654.561389,786.826111,1.092814
1,2358,29.547778,1,3,0.0,169.216,15.791680,7.601939,132.9952,0.017045,0,1,0,12.272500,204.958611,0.284665
2,3808,92.468333,0,1,0.0,139.416,23.996349,49.585790,11.6404,0.584983,0,1,0,421.187778,23.906667,0.033204
3,4101,44.947917,0,3,1.0,158.080,12.843435,6.393176,63.7240,0.068463,0,1,0,49.293056,25.586389,0.035537
4,4434,12.255000,1,2,1.0,129.446,27.704460,25.895974,12.0428,0.086622,0,1,0,62.367500,10.373611,0.014408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
868,14544270,34.933333,0,3,0.0,178.098,12.300091,10.010908,111.1168,2.213419,0,1,0,1593.661944,321.138889,0.446026
869,14671923,76.025139,0,3,0.0,164.736,8.703396,7.231071,16.5840,0.233212,0,1,0,167.912778,169.050000,0.234792
870,14779650,88.589722,0,3,0.0,176.236,10.573623,6.285431,26.7112,0.088035,0,1,0,63.385556,26.188056,0.036372
871,15279967,65.406944,0,3,0.0,148.248,9.275948,9.987104,54.9224,0.076142,0,1,0,54.822222,65.405278,0.090841


In [0]:
# Indices for features
pd.DataFrame({'features': user_time_table.columns})

,features
0,userId
1,user_median
2,num_recent_workouts
3,recent_sport
4,workout_soon
5,recent_heart_rate
6,recent_speed
7,recent_distance
8,recent_altitude
9,months_to_next1


# Metrics

In [0]:
# Feature Selection

# Best results: 0, 5, 6, 8, 10, 12, 14
# All the features: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18

features = [0, 1, 6, 7, 13] # 11, 8

x = user_time_table.iloc[:, features].values
y = user_time_table.iloc[:, 4].values

In [0]:
# Split data into training and testing (80, 20)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [0]:
# Standardize data

from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
x_train = sc_x.fit_transform(x_train)
x_test = sc_x.transform(x_test)

# SVM (RBF) Model
#### Accuracy: 79.4%

In [0]:
# Fit SVM Model
# Features: 0, 1, 6, 7, 13 -> 75.4% Accuracy

from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0, C=1.6) # The best key value was found to be 1.6
classifier.fit(x_train, y_train)

SVC(C=1.6, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=False)

In [0]:
# Predictions

y_pred = classifier.predict(x_test)

In [0]:
# SVM (RBF) Model Accuracy

from sklearn.metrics import confusion_matrix
print("SVM (RBF) Accuracy: " + str((y_test == y_pred).sum() / (len(x_test)) * 100) + "%")

SVM (RBF) Accuracy: 73.71428571428571%


# Linear SVM Model
#### Accuracy: 81.1 %

In [0]:
# Fit SVM Model

from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(x_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=0, shrinking=True, tol=0.001,
    verbose=False)

In [0]:
# Predictions

y_pred = classifier.predict(x_test)

In [0]:
# SVM (Linear Kernel) Model Accuracy

from sklearn.metrics import confusion_matrix
print("SVM (Linear Kernel) Accuracy: " + str((y_test == y_pred).sum() / (len(x_test)) * 100) + "%")

SVM (Linear Kernel) Accuracy: 71.42857142857143%


# KNN Model
#### Accuracy: 77.7%

In [0]:
# Fit KNN Model

from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2)
classifier.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [0]:
# Predicitions

y_pred = classifier.predict(x_test)

In [0]:
# KNN Model Accuracy

from sklearn.metrics import confusion_matrix
print("KNN Accuracy: " + str((y_test == y_pred).sum() / (len(x_test)) * 100) + "%")

KNN Accuracy: 70.28571428571428%


# Logistic Regression
#### Accuracy: 80.0%

In [0]:
#@title Logistic Regression Model


In [0]:
# Fit Linear Regression model 
# Features: 0, 1, 6, 7, 8, 11, 13 -> 74.3?


from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state=0)
classifier.fit(x_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
# Correlation coefficient for selected features

coef_data = {'index': features, 'coef' : classifier.coef_.tolist()[0]}
coef_df = pd.DataFrame(coef_data)
coef_df['coef'] = coef_df['coef']**2 # get rid of negatives
coef_df.sort_values(by=['coef'])

,index,coef
4,8,0.001043
5,11,0.023173
3,7,0.051392
2,6,0.058504
0,0,0.088809
6,13,0.141633
1,1,0.328947


In [0]:
# Predictions

y_pred = classifier.predict(x_test)

In [0]:
# Logistic Regression Model Accuracy

print("Logistic Regression Accuracy: " + str((y_test == y_pred).sum() / (len(x_test)) * 100) + "%")

Logistic Regression Accuracy: 73.71428571428571%


# Baseline Models
#### Accuracy1 (check if each user has a median time gap <= the median time gap of all users) ---> 68.6%


#### Accuracy2 (see if the second to last workout was in the median time gap) ---> 67.9%

In [0]:
#Our Check: check if each user has a median time gap <= the median time gap of all users

median_gap = 1.9372280092592593


count = 0
for i in range(len(user_time_table['workout_soon'])):
 if user_time_table['user_median'][i] <= median_gap and user_time_table['workout_soon'][i] == 1:
   count += 1
 if user_time_table['user_median'][i] > median_gap and user_time_table['workout_soon'][i] == 0:
   count += 1
print('Baseline Percent Accuracy: ' + str((count/len(user_time_table)) * 100) + '%')

Baseline Percent Accuracy: 68.61397479954181%


In [0]:
#McAuley Check: see if the second to last workout was in the median time gap

count = 0
for i in range(len(user_time_table['workout_soon'])):
 if user_time_table['hrs_to_next1'][i] <= median_gap and user_time_table['workout_soon'][i] == 1:
   count += 1
 if user_time_table['hrs_to_next1'][i] > median_gap and user_time_table['workout_soon'][i] == 0:
   count += 1
print('Baseline Percent Accuracy: ' + str((count/len(user_time_table)) * 100) + '%')

Baseline Percent Accuracy: 67.92668957617411%
